In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata
from  geopy.geocoders import Nominatim

In [5]:
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 2000)

In [6]:
data = pd.read_table('football-city-html.txt') 

In [7]:
df = data.rename(columns={"</div>": "data"})

In [8]:
terms = ['col-sm-6 col-xs-6', 'col-sm-3 hidden-xs', '/modules/images/flags/']

In [9]:
df = df[df.data.str.contains('|'.join(terms))] 

In [10]:
df.drop(index=5, inplace=True)

In [11]:
df.shape

(5782, 1)

In [12]:
pattern = '|'.join(['<div class=\"col-sm-6 col-xs-6\"><b>', '<img src=\"/modules/images/flags/_', '<div class=\"col-sm-3 hidden-xs\">', '</b>', '</div>'])

In [13]:
df.data = df.data.str.replace(pattern, '')

/home/sergey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [15]:
def removing_patterns(df: pd.DataFrame, col_names: list, patern: str):
    df[col_names].apply(lambda x: str(x) if re.search(pattern, str(x)) is None else re.findall(pattern, str(x)))
    return df

In [16]:
pattern = r'\"[\s|\w]+\">$'
df = removing_patterns(df, list(df), pattern)

In [17]:
find_symols = r'[^.a-zA-Z0-9)(&"\s>;<=,-/}{\']'

In [18]:
rare_symbols = df.data.apply(lambda x: ''.join(re.findall(find_symols, str(x))))

In [19]:
str(set(rare_symbols))

"{'', 'ığ', 'çã', 'ø', 'é', 'ġāī', 'łó', 'ăăş', 'ğ', 'ó', 'ã', 'çöüü', 'îş', 'Š', 'É', 'Žž', 'İ', 'İÖİ', 'íáé', 'óú', 'ô', 'ï', 'Í', 'ń', 'î', 'ęó', 'íá', 'ë', 'ãç', 'žíš', 'ľé', 'ıçğ', 'ýá', 'çş', 'Ñú', 'ðæ', '’', 'áž', 'ūẓī', 'ľč', 'Łóź', 'ä', 'Şüə', 'äé', 'ú', 'łł', 'õ', 'ł', 'éḥ', 'Şı', 'šéá', 'óñ', 'žá', 'ủầộ', 'å', 'ëçë', 'ööö', 'šýš', 'ę', 'üü', 'ãã', 'áó', 'ş', 'ī', 'âş', 'ří', 'â', 'ü', 'ı', 'áň', 'ā’', 'Ō', 'öö', 'šá', 'şş', 'È', 'à', 'ď', 'áŠ', 'čŠ', 'á', 'ç', 'ñ', 'ăă', 'Čč', 'Ž', 'ėž', 'ññ', 'éšě', 'šňé', 'º', 'çë', 'óŁę', 'əə', 'ľý', 'éï', 'óã', 'éé', 'í', 'íí', 'úí', 'ă', 'âá', 'ľ', 'đ', 'āḍ', 'óá', 'ţ', 'ýáš', 'ěříž', 'áá', 'əəə', 'ā', '’éé', 'ýí', 'öð', 'ň', 'ëë', 'Ħ', 'éë', 'íš', 'ū', 'ýŽáš', 'Łę', 'ž', 'áš', 'š', 'ážá', 'áé', 'öä', 'çğ', 'æ', 'ș', 'éá', 'íð', 'Śś', 'āā', 'ăţ', 'ą', 'ðø', 'àộ', 'è', 'ží', 'ō', 'ř', 'č', 'øø', 'ć', 'ăăăş', 'àò', 'ış', 'ź', 'áí', 'ê', 'ö', 'ïé', 'žň', 'ãí', 'ôô', 'ää', 'šć', 'ýŽ', 'čí', 'Å', 'ė', 'šáá', 'ʿā', 'óó', 'ééá', 'óę', 'Šľ', 'ġ

In [20]:
all_symbols = []
for symbol in rare_symbols:
    symbols = ''.join(symbol)
    all_symbols.append(symbols)
all_symbols = str(all_symbols)
all_symbols = ''.join(set(str(all_symbols)))
all_symbols = all_symbols.replace(' ', '')
all_symbols = all_symbols.replace(']', '')
all_symbols = all_symbols.replace('[', '')
all_symbols = all_symbols.replace(',', '')
all_symbols = all_symbols.replace('\'', '')

In [21]:
all_symbols

'øéğóãŠÉÖİôộïÍńîëḍ’śäúõłòåęşīâüıŌÈàďáẓçñŽºġíăľđŞţāňĦūŁðžšŚʿæČșąèḥủōřÑčćźêöəÅėěầý'

In [22]:
pattern_2 = r'^[0-9]+.png'
pattern_3 = r'title=\"(.+?)\"'

In [24]:
df = df.data.apply(lambda x: ''.join(re.findall(pattern_3, str(x)) if re.search(pattern_2, str(x)) is not None else x))

In [25]:
df = pd.DataFrame(df)

In [26]:
df.shape

(5782, 1)

In [27]:
df.iloc[648:649,:]

,data
3067,


In [28]:
df.data.replace('', np.nan, inplace=True)

In [29]:
df.dropna(subset=['data'], inplace=True)

In [30]:
df.shape

(5757, 1)

In [31]:
def str_decode(str_obj: str):
    normalized = unicodedata.normalize('NFD', str_obj)
    decode_str = u"".join([c for c in normalized if not unicodedata.combining(c)])
    return decode_str

In [32]:
df.data = df.data.apply(lambda x: str_decode(x))

In [33]:
rare_symbols_keys = {'ł':'l','Ł':'L','º':'','ʿ':'\'','ə':'e','Ħ':'H','đ':'d','’':'\'','ı':'i','æ':'a','ð':'d','ø':'o'}

def replace_rare_symbols(df: pd.DataFrame, col_name: str, dict_obj: dict):
    for k, v in rare_symbols_keys.items():
        df[col_name] = df[col_name].apply(lambda x: x.replace(k, v) if re.search(k, str(x)) else x)
    return df[col_name]       

In [34]:
df.data = replace_rare_symbols(df, 'data', rare_symbols_keys)

In [35]:
df

,data
13,AaB
18,Denmark
20,Aalborg
27,Aalesund
32,Norway
...,...
27134,Switzerland
27136,Bern
27143,Young Buffaloes
27148,Swaziland


In [36]:
df.reset_index(inplace=True)

In [37]:
df.drop('index', axis=1, inplace=True)

In [38]:
club, country, city = df[0::3], df[1::3], df[2::3]

In [39]:
table_df = pd.concat([club.reset_index(drop=True), country.reset_index(drop=True), city.reset_index(drop=True)], axis=1)

In [40]:
table_df.columns.values[0] = "club"
table_df.columns.values[1] = "country"
table_df.columns.values[2] = "city"

In [41]:
table_df.shape

(1919, 3)

In [42]:
df = table_df.copy()

In [43]:
# take city.values from hooks for transcription
df.city = df.city.map(lambda x: ''.join(re.findall(r'\((.*?)\)', x)) if re.search(r'\(', x) else x)

In [44]:
# remove Buenos Aires districts
buenos_pattern = 'Buenos'
df.city = df.city.apply(lambda x: x if re.search(buenos_pattern, str(x)) is None else 'Buenos Aires')

In [45]:
# take city.values before '/'
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?)/', x)) if re.search(r'\/', x) else x)

In [46]:
# take city.values before ',' without districts
df.city = df.city.map(lambda x: ''.join(re.findall(r'^(.*?),', x)) if re.search(r',', x) else x)

In [47]:
# rename 'D.F.' at 'Mexico'
mexico_pattern = 'D.F.'
df.city = df.city.apply(lambda x: x if re.search(mexico_pattern, str(x)) is None else 'Mexico')

In [49]:
df.head(3)

,club,country,city
0,AaB,Denmark,Aalborg
1,Aalesund,Norway,Alesund
2,Aarau,Switzerland,Aarau


In [50]:
df.tail(3)

,club,country,city
1916,Young Africans,Tanzania,Dar-es-Salaam
1917,Young Boys,Switzerland,Bern
1918,Young Buffaloes,Swaziland,Manzini
